**2021/22**

# Recommender Systems
This lecture is about recommender systems (or recommendation systems). In the meantime, we highlight the usefulness of Spark SQL, particularly when it relates to persistent tables.

# Spark SQL

As mentioned in the initial lectures, Spark SQL is a Spark module for structured data processing. It works alongside the APIs of DataFrame and Dataset and it is responsible for performing extra optimizations. We can also execute SQL queries and reading data from various files formats an Hive tables. (Apache Hive can manage large datasets residing in distributed storage using SQL)

Further details can be found in https://spark.apache.org/docs/latest/sql-programming-guide.html  and https://spark.apache.org/docs/latest/api/sql/index.html

We can check the reference guide for Structured Query Language (SQL) which includes syntax, semantics, keywords, and examples for common SQL usage.

# Problem formulation

This exercise aims to build a recommender system of books, with focus on the recommendation model
itself.
The functional requirements for the Spark program we want to create are as follows:
1. To load the dataset and perform exploratory analysis, then store the information properly cleaned, including as SQL tables.
2. To create a recommendation model supported by the ALS algorithm provided by Spark MLlib.
3. To pre-compute recommendations and store them in SQL tables.
4. To show recommendations.


**Dataset**


The data we are processing is from the dataset **Book-Crossing**. As stated in the website from where it can be downloaded, http://www2.informatik.uni-freiburg.de/~cziegler/BX/ , the BookCrossing (BX) dataset was collected by Cai-Nicolas Ziegler in a 4-week crawl (August / September 2004) from the Book-Crossing community with kind permission from Ron Hornbaker, CTO of Humankind Systems. It contains 278,858 users (anonymized but with demographic information) providing 1,149,780 ratings (explicit / implicit) about 271,379 books.

Alternatively, we can use the command *wget* from the Terminal to download the dataset:

    wget http://www2.informatik.uni-freiburg.de/~cziegler/BX/BX-CSV-Dump.zip


The dataset comprises 3 tables, as follows:
- **BX-Users**. Contains the users. Note that user IDs ( User-ID ) have been anonymized and map to integers. Demographic data is provided ( Location , Age ) if available. Otherwise, these fields contain NULL-values.
- **BX-Books**. Books are identified by their respective ISBN. Invalid ISBNs have already been removed from the dataset. Moreover, some content-based information is given ( Book-Title , Book-Author , Year-Of-Publication , Publisher ), obtained from Amazon Web Services. Note that in case of several authors, only the first is provided. URLs linking to cover images are also given, appearing in three different flavours ( Image-URL-S , Image-URL-M , Image-URL-L ), i.e., small, medium, large. These URLs point to the Amazon web site.
- **BX-Book-Ratings**. Contains the book rating information. Ratings ( Book-Rating ) are either explicit, expressed on a scale from 1-10 (higher values denoting higher appreciation), or implicit, expressed by 0.
The columns are separated by ; and all files contain the correspondent header.


In [ ]:
# If we need to install some packages, e.g. matplotlib

# ! pip3 install matplotlib
# ! pip3 install seaborn

In [ ]:
# Some imports 

import os 

import numpy as np 
import pandas as pd  
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

# Useful visualization functions

Some functions that we can use to plot data but as Python dataframes.

**Disclaimer**: these functions are broadly distributed among users. Further adjustments are needed and/or advisable. Feel free to use your own plotting functions.

In [ ]:
def plotHistogram(df, xcol, huecol, bins):
    if huecol:
        sns.histplot(data=df, x=xcol, hue=huecol, multiple="stack")
    else:
        sns.histplot(data=df, x=xcol, bins=bins)

# Collect and label data

## Data ingestion

In [ ]:
! pwd 
! ls -la

In [ ]:
! head -n 2 BX-Users.csv
! tail -n 2 BX-Users.csv

In [ ]:
! head -n 2 BX-Books.csv
! tail -n 2 BX-Books.csv

In [ ]:
! head -n 2 BX-Book-Ratings.csv
! tail -n 2 BX-Book-Ratings.csv

In [ ]:
# some Spark related imports we will use hereafter

import sys

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

from pyspark.ml import Pipeline

from pyspark.ml.feature import StringIndexer
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
# Build a SparkSession instance if one does not exist. Notice that we can only have one per JVM

spark = SparkSession\
    .builder\
    .appName("Recommender")\
    .config("spark.sql.shuffle.partitions",6)\
    .config("spark.sql.repl.eagerEval.enabled", True)\
    .getOrCreate()


In [ ]:
# Read the dataset 

df_raw_users = spark.read.csv("BX-Users.csv", header="true", 
                              inferSchema="true", sep=";")

df_raw_books = 

df_raw_ratings = 


## Columns to keep

In [ ]:
# Check users - schema and count

df_raw_users.printSchema()
df_raw_users.show(2, vertical=True, truncate=False) 
num_users = df_raw_users.count()
num_users

In [ ]:
# Check books - schema and count

df_raw_books.

num_books

In [ ]:
# Check ratings - schema and count

df_raw_ratings.

num_ratings

There are no reasons to drop any column.

# Evaluate data

Let us get some data insight, with some exploratory data analysis based on descriptive statistics and visualizations if advisable.

In [ ]:
# Check some column statistics, one by one, using describe

for cl in df_raw_users.columns:
    
    

In [ ]:

for cl in df_raw_books.columns:
    

In [ ]:

for cl in df_raw_ratings.columns:
    

Following previous understanding, all collected data should be considered as of interest.

# Feature Engineering

Now we have to prepare data in a way that it can be properly used by ML algorithms, which includes selection and extraction of features, as well as dealing with poor data quality if that is the case.

## Data cleasing

We will look at
* Data types
* Nulls
* Extreme values e.g. outliers

In [ ]:
[num_users, df_raw_users.dropna().count()]

In [ ]:
[num_books, 

In [ ]:
[num_ratings, 

Only differences are spotted for users.

In [ ]:
df_raw_users.filter(column("Age").isNull()).show()

At this point, we can conclude that:

- User-ID is set as string in users but integer in ratings
- Age is set as string in users, with range of values between 0 to null
- Year-Of_Publication ranges from 0 to 2050
- Book-rating ranges from 0 to 10
- Only two observations in users hold null values

What can we do now about nulls, data types or extreme values? 

Recall that if we delete an observation in one table, still consistency among tables has to be preserved. We leave it as exercise.

In [ ]:
# Outliers: for that, we use summary(), one column by one, using summary

for cl in df_raw_users.columns:
    
        

In [ ]:
for cl in df_raw_books.columns:
    
    

In [ ]:
for cl in df_raw_ratings.columns:
    
    

In [ ]:
 df_raw_users.select(col("Age")).where(col("Age") == 'NULL').count()

There are a lot of NULLs (as string) in the Age column. So, we may
- replace the NULLs with the average of others for example (with Imputer) 
- drop the column Age in case we can live without it 
- delete the records with NULL in the column Age

It is for further discussion!


In [ ]:
 # We drop column Age now
    
df_raw_users = 

Let us carry out further checking.

In [ ]:
# 2004 was when data was collected

( df_raw_books
     .select('Year-Of-Publication')
     .where(col('Year-Of-Publication')>2004)
     .distinct()
     .orderBy('Year-Of-Publication')
     .show()
)

In [ ]:
# prior to 1900

( df_raw_books
     .select('Year-Of-Publication')
     
 
)

In [ ]:
# to plot year of publication

df_plot = df_raw_books.

In [ ]:
plotHistogram(df_plot, 'Year-Of-Publication', "", 4)

In [ ]:
# We will drop some columns anyway

df_raw_books = df_raw_books.drop('Year-Of-Publication', 
                         'Image-URL-S', 'Image-URL-M', 'Image-URL-L')

## Saving clean data

As usual, we may want to have a smaller dataset just for the purpose of testing locally.
But in this case, as mentioned above, consistency among the three tables has to be guaranteed. 

Let us try to use just the normal dataset.

In [ ]:
# Users

output_users = "users.parquet"
df_raw_users.write.mode("overwrite").parquet(output_users)

In [ ]:
# Books



In [ ]:
# Ratings



In [ ]:
# Check in the running directory if that was accomplished

! ls -la

Also, save them as persistent tables into Hive metastore

Notice
- An existing Hive deployment is not necessary to use this feature. Spark will take care of it.
- We can create a SQL table from a DataFrame with createOrReplaceTempView command, valid for the session. (there is also the option of global temporary views, to be shared among all sessions till the Spark application terminates)
- But with saveAsTable, there will be a pointer to the data in the Hive metastore. So persistent tables will exist even after the Spark program has restarted, as long as connection is maintained to the same metastore.

See details in http://spark.apache.org/docs/latest/sql-data-sources.html


In [ ]:
# Persistent tables into Hive metastore

df_raw_users.write.mode("overwrite").saveAsTable("UsersTable")
df_raw_books.
df_raw_ratings.

## Data to be used hereafter

In [ ]:
# Instead of a smaller dataset, once properly built

# df_clean_users = ...
df_clean_users = df_raw_users
df_clean_books = df_raw_books
df_clean_ratings = df_raw_ratings


In [ ]:
# Delete memory consuming variables that are no longer needed

del 


## Final  overview
After establishing the clean data to be used, we should get an overview about what we have achieved, with some statistics and visualizations.

**But** 

we leave it as it is now, because so far there are no significant changes (we just drop columns). Eventually, we could check the ratings and draw some plots, as it is the critical part of the system. You can have a go in that regard.


## Features transformation

As mentioned, ratings are critial here. Recall that, in the dataframe, the schema is User-ID (integer), ISBN (string) and Book-rating (integer). ISBN poses a problem as the ML algorithm requires numbers to process. Hence, we have to convert it to numbers - we will use `StringIndexer` to do so.


In [ ]:
# StringerIndexer for ISBN

indexer = StringIndexer(inputCol="ISBN", outputCol="ISBN-Index", handleInvalid="keep") 


In [ ]:
# Columns from ratings that are going to be considered in the model

user_col = "User-ID"
item_col = "ISBN-Index" 
rating_col = "Book-Rating"

# Select and train model

In order to create the recommendation model, we will use the Alternating Least Squares (ALS) algorithm provided by Spark MLlib. See details in http://spark.apache.org/docs/latest/ml-collaborative-filtering.html , as we advise to check the main assumptions the implemented algorithm relies upon. For example, notice that:
- it underlies a collaborative filtering strategy;
- it aims to fill in the missing entries of a user-item association matrix, in which users and items are described by a small set of latent factors that can be used to predict missing entries. The latent factors are learned by the ALS algorithm.

Again, as for data to train the model, the focus is on ratings.

## Train/test split

We will use the standard split 80/20, for the reasons explained in previous lectures.

In [ ]:
# train/test clean ratings split

df_train, df_test = 

# caching data ... but just the training part
df_train

# print the number of rows in each part
print(f"There are {df_train.count()} rows in the training set and {df_test.count()} in the test set.")


**Notice** 

As we did with clean data, we may consider storing the data split into files, should we want to use it elsewhere. 
This relates to the need of guaranteeing unicity in a different environment. 
We leave it as it is now.

## ALS model

Using the `ALS` estimator (the algorithm) to learn from the training data and consequently to build the model. 

In [ ]:
# Build the recommendation model using ALS on the training data
# note that we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
        
        
als = ALS(maxIter=5, regParam=0.01, 
          userCol=user_col, 
          itemCol=item_col, 
          ratingCol=rating_col,
          coldStartStrategy="drop",
          implicitPrefs=True
         )

# if the rating matrix is derived from another source of information
# (i.e. it is inferred from other signals), we may set implicitPrefs
# to True to get better results (see ALS reference)



## ML pipeline configuration

In [ ]:
# The pipeline holds two stages set above

# As we will see below, we are going to use it just for evaluation purposes

pipeline = Pipeline(stages=[indexer, als])


## Model fitting
Get the model (as transformer) by fitting the pipeline to training data.

In [ ]:
pipeline_model = pipeline.fit(df_train)


# Evaluate model

Let us evaluate the ALS model.

## Testing the model

It is time to apply the model built to test data. Again, we will use the pipeline set above. Notice that, since the pipeline model is a transformer, we can easily apply it to test data.

In [ ]:
# Make predictions on test data and show values of columns of interest

df_prediction = pipeline_model.transform(df_test)


In [ ]:
# Checking its schema and content

df_prediction. 


In [ ]:
# Show predictions ordered by USER-ID

df_prediction.orderBy("User-ID").show(truncate=False)

In [ ]:
# Show predictions ordered by ISBN-Index

df_prediction.

## Evaluation metrics

Let us use an evaluator.


In [ ]:
# Evaluate the model by computing the RMSE on the test data

evaluator = RegressionEvaluator(metricName="rmse",
                                labelCol=rating_col,
                                predictionCol="prediction")

rmse = evaluator.evaluate(df_prediction)
print("Root-mean-square error = " + str(rmse))

## Saving the pipeline

In [ ]:
# We can save the pipeline for further use should it be required

pipeline.save("pipeline-ALS")

# later on, it can be loaded anywhere


In [ ]:
! ls -la

In [ ]:
! ls -la pipeline-ALS

## Pre-computing recommendations and storing as persistent tables

The `ALS` algorithm provides some functions to get recommendations directly. 

Although we can achieve results if working with predictions after the pipeline set (see below), we will take advantage of such methods directly. We should emphasize that, as it stands, we will not be using the pipeline for this task.


In [ ]:
# Checking with training data for the sake of example

df_train_indexed = indexer.fit(df_train).transform(df_train)
model = als.fit(df_train_indexed)

In [ ]:
# Get all distinct users and books

#user_col = "User-ID"
#item_col = "ISBN-Index" 
#rating_col = "Book-Rating"

users = df_train_indexed.select(als.getUserCol()).distinct()

books = df_train_indexed.select(als.getItemCol()).distinct()

In [ ]:
users.show()

In [ ]:
books.show()

In [ ]:
[users.count(), books.count()]

In [ ]:
# Generate top book recommendations for users

top_n_books = 2
user_recs = model.recommendForAllUsers(top_n_books)

# Generate top book recommendations for a specified set of users

# subset_users = users.limit(5)
# user_subset_recs = model.recommendForUserSubset(subset_users, top_n_books)


In [ ]:
user_recs.show(truncate=False)

# user_subset_recs.show(truncate=False)


In [ ]:
# Generate top user recommendations for each book

top_n_users = 2
book_recs = model.recommendForAllItems(top_n_users)

# Generate top user recommendations for a specified set of books

# subset_books = books.limit(5)
# book_subset_recs = model.recommendForItemSubset(subset_books, top_n_users)


In [ ]:
book_recs.show(truncate=False)

# book_subset_recs.show(truncate=False)


Save the recommendations as persistent tables into the Hive metastore.

In [ ]:
 user_recs.write.mode("overwrite").saveAsTable("UserRecommendationsTable")


In [ ]:
book_recs.write.mode("overwrite").saveAsTable("BookRecommendationsTable")


In [ ]:
! ls -la

## Exploring results
1. Given a user, shows the recommended list of books.
2. Given a book, shows the list of users who might be interested on.

We are going to use Spark SQL tables.

In [ ]:
# user to explore

user = 0


In [ ]:
# book to explore

book = 0


First, let us check the SQL tables.

In [ ]:
# Register information about users as a SQL temporary view

df_clean_users.createOrReplaceTempView("users")


In [ ]:
# Register information about books as a SQL temporary view

df_clean_books


In [ ]:
print(spark.catalog.listDatabases())

In [ ]:
 spark.catalog.listTables(dbName="default")

In [ ]:
# Use managed tables

spark.sql("USE default")


In [ ]:
spark.catalog.listColumns('bookstable')


In [ ]:
 spark.sql("SELECT * FROM users").show(10, truncate=False)

In [ ]:
 spark.sql("SELECT * FROM books").show(10, vertical=True, truncate=False)

In [ ]:
 print("The recommended books for user " + str(user) + " are: ")

We leave it as exercise!

In [ ]:
 print("The users who might be interested on the book " + str(book) + " are: ")

**We leave it as exercise!**

# Tune model

We can improve the model. For example, by carrying out better data cleasing operations and take into consideration efficiency issues. 


# Additional exercise

Given the current status of this notebook, redo its content such that major tasks are split into 
various notebooks, ou Python modules. 
The purpose is to modularize code having in mind the setup of a real recommender system. That is:
- A downloader module, focussing on downloading data, cleasing it, and then storing it in a data store.
- A recommender module, to create a recommendation module and to pre-compute recommendations in order to save them a data store.
- A recommender server, to retrieve recommendations upon queries made to the data store.

# References

* Learning Spark - Lightning-Fast Data Analytics, 2nd Ed. J. Damji, B. Wenig, T. Das, and D. Lee. O'Reilly, 2020
* Spark: The Definitive Guide - Big Data Processing Made Simple, 1st Ed. B. Chambers and M. Zaharia. O'Reilly, 2018
* http://spark.apache.org/docs/latest/ml-guide.html
* https://docs.python.org/3/ 
